DATA 601 Assignment 4

Evan Losier 30022571

In [52]:
# Part 1

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import re

# Load the dataset
df = pd.read_csv("Building_Energy_Benchmarking.csv")

# Display shape, columns, and data types
display(df.shape)
display(df.columns)
display(df.dtypes)

# Count of missing data
nulls = df.isnull().sum()
display(nulls)

# Drop columns <60% non-null
df.dropna(axis=1, thresh=np.ceil(0.6 * len(df.index)), inplace=True)

# Remove commas from text-based numeric types and convert column type to float
for colName in df.select_dtypes(include=['object']).columns[6:-1]:
    df[colName] = [re.sub(',', '', str(x)) for x in df[colName]]
    df[colName] = df[colName].astype('float64')

# Impute missing values
df.fillna({
    'Weather Normalized Source Energy Use (GJ)': df['Weather Normalized Source Energy Use (GJ)'].median(),
    'Natural Gas Use (GJ)': df['Natural Gas Use (GJ)'].median()
}, inplace=True)

# Standardize postal codes to have uppercase letters and a space in the middle
oldPost = r"([a-zA-Z][0-9][a-zA-Z]) *([0-9][a-zA-Z][0-9])"
newPost = r"\1 \2"
df['Postal Code'] = [re.sub(oldPost, newPost, str(x).upper()) for x in df['Postal Code']]

# Property Names and Addresses do not contain any meaningless information. I don't see what needs to be cleaned here.

# Postal Codes, Property Names, and Addresses cannot be converted to numerical types.


(494, 31)

Index(['Property Id', 'Property Name', 'Address 1', 'City', 'Postal Code',
       'Province', 'Primary Property Type - Self Selected',
       'Number of Buildings', 'Year Built',
       'Property GFA - Self-Reported (m²)', 'ENERGY STAR Score',
       'Site Energy Use (GJ)', 'Weather Normalized Site Energy Use (GJ)',
       'Site EUI (GJ/m²)', 'Weather Normalized Site EUI (GJ/m²)',
       'Source Energy Use (GJ)', 'Weather Normalized Source Energy Use (GJ)',
       'Source EUI (GJ/m²)', 'Weather Normalized Source EUI (GJ/m²)',
       'Total GHG Emissions (Metric Tons CO2e)',
       'Total GHG Emissions Intensity (kgCO2e/m²)',
       'Direct GHG Emissions (Metric Tons CO2e)',
       'Direct GHG Emissions Intensity (kgCO2e/m²)',
       'Electricity Use - Grid Purchase (kWh)', 'Natural Gas Use (GJ)',
       'District Hot Water Use (GJ)',
       'Electricity Use – Generated from Onsite Renewable Systems (kWh)',
       'Green Power - Onsite and Offsite (kWh)',
       'Avoided Emissions - Ons

Property Id                                                                int64
Property Name                                                             object
Address 1                                                                 object
City                                                                      object
Postal Code                                                               object
Province                                                                  object
Primary Property Type - Self Selected                                     object
Number of Buildings                                                        int64
Year Built                                                                 int64
Property GFA - Self-Reported (m²)                                         object
ENERGY STAR Score                                                        float64
Site Energy Use (GJ)                                                      object
Weather Normalized Site Ener

Property Id                                                                0
Property Name                                                              0
Address 1                                                                  0
City                                                                       0
Postal Code                                                                0
Province                                                                   0
Primary Property Type - Self Selected                                      0
Number of Buildings                                                        0
Year Built                                                                 0
Property GFA - Self-Reported (m²)                                          0
ENERGY STAR Score                                                        329
Site Energy Use (GJ)                                                       0
Weather Normalized Site Energy Use (GJ)                                    0

In [66]:
# Part 2

# Show summary statistics
display(df.describe())

# The main observation to be drawn from the summary statistics is that
# all the columns related to property metrics such as area, energy,
# and emissions have much larger means than medians. This means the data
# has many outliers with high metrics in all categories.

# Average Energy Use Intensity by Property Type
meanEUI = df.groupby('Primary Property Type - Self Selected').agg(meanEUI=('Site EUI (GJ/m²)', 'mean'))
display(meanEUI)

# Total Greenhouse Gas emissions by year
sumGHG = df.groupby('Year Ending').agg(meanEUI=('Total GHG Emissions (Metric Tons CO2e)', 'sum'))
display(sumGHG)

# Top 5 properties with the highest total energy consumption
sumEnergy = df.groupby('Property Id').agg(totalEnergyUse=('Site Energy Use (GJ)', 'sum'))
t5sumEnergy = sumEnergy.nlargest(5, 'totalEnergyUse')
display(t5sumEnergy)

# Values that didn't conform to expected numeric formats were corrected in part 1

# Detect Total GHG Emissions outliers using IQR method
q1 = df['Total GHG Emissions (Metric Tons CO2e)'].quantile(0.25)
q3 = df['Total GHG Emissions (Metric Tons CO2e)'].quantile(0.75)
iqr = q3 - q1
lower = q1 - (1.5 * iqr)
upper = q3 + (1.5 * iqr)
outliers = df[(df['Total GHG Emissions (Metric Tons CO2e)'] < lower) | (df['Total GHG Emissions (Metric Tons CO2e)'] > upper)]
display(outliers)
display(lower)
display(upper)

# Replace GHG emission outliers with the median value for their property type
medianByType = df.groupby('Primary Property Type - Self Selected').agg(medianGHGbyType=('Total GHG Emissions (Metric Tons CO2e)', 'median'))
df = pd.merge(df, medianByType, on='Primary Property Type - Self Selected', how='left')
df['Total GHG Emissions (Metric Tons CO2e)'] = np.where(
    (df['Total GHG Emissions (Metric Tons CO2e)'] < lower) |
    (df['Total GHG Emissions (Metric Tons CO2e)'] > upper),
    df['medianGHGbyType'], df['Total GHG Emissions (Metric Tons CO2e)'])
df.drop('medianGHGbyType', axis=1, inplace=True)

,Property Id,Number of Buildings,Year Built,Property GFA - Self-Reported (m²),Site Energy Use (GJ),Weather Normalized Site Energy Use (GJ),Site EUI (GJ/m²),Weather Normalized Site EUI (GJ/m²),Source Energy Use (GJ),Weather Normalized Source Energy Use (GJ),Source EUI (GJ/m²),Weather Normalized Source EUI (GJ/m²),Total GHG Emissions (Metric Tons CO2e),Total GHG Emissions Intensity (kgCO2e/m²),Direct GHG Emissions (Metric Tons CO2e),Direct GHG Emissions Intensity (kgCO2e/m²),Electricity Use - Grid Purchase (kWh),Natural Gas Use (GJ),Year Ending
count,4.940000e+02,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,4.940000e+02,494.000000,494.000000
mean,1.308877e+07,1.060729,1980.091093,4752.556883,8265.670040,8397.193117,1.774798,1.810324,10590.527733,10221.438866,2.282753,2.319717,425.726670,158.671660,281.868826,63.821862,6.011028e+05,5520.693522,2020.995951
std,5.659556e+06,0.278281,25.159568,10128.320688,19733.748811,19877.336939,1.306700,1.331529,23438.866328,22866.650000,1.597846,1.622897,803.934917,109.472637,884.613506,56.835735,1.184414e+06,17218.457051,1.414208
min,6.169481e+06,1.000000,1896.000000,113.000000,141.000000,146.000000,0.060000,0.060000,237.000000,146.000000,0.110000,0.110000,15.000000,10.200000,0.000000,0.000000,1.076580e+04,3.000000,2019.000000
25%,9.563763e+06,1.000000,1970.000000,1096.650000,1232.825000,1256.150000,0.990000,1.000000,1601.525000,1562.825000,1.312500,1.350000,113.850000,93.100000,43.000000,33.325000,1.074446e+05,861.050000,2020.000000
50%,9.997794e+06,1.000000,1978.000000,1806.750000,2555.650000,2572.500000,1.290000,1.310000,3238.600000,3144.800000,1.680000,1.690000,228.400000,117.600000,78.700000,43.900000,2.237719e+05,1569.750000,2021.000000
75%,2.198860e+07,1.000000,1996.000000,4257.900000,7999.050000,8006.575000,2.065000,2.110000,10242.400000,9637.275000,2.622500,2.652500,541.500000,186.975000,247.425000,74.050000,5.492725e+05,4816.200000,2022.000000
max,2.198863e+07,3.000000,2018.000000,85941.000000,243202.000000,242611.000000,8.430000,8.360000,261481.000000,260399.000000,10.130000,10.060000,7768.200000,666.200000,12243.000000,386.000000,9.786193e+06,238415.000000,2023.000000


,meanEUI
Primary Property Type - Self Selected,
Distribution Center,3.286000
Fire Station,1.208827
Fitness Center/Health Club/Gym,4.385000
Heated Swimming Pool,4.805333
Ice/Curling Rink,2.182200
Indoor Arena,1.106000
Mixed Use Property,0.458000
Museum,1.584000
Non-Refrigerated Warehouse,0.768000


,meanEUI
Year Ending,
2019,44516.450
2020,42133.000
2021,41271.275
2022,41655.125
2023,40733.125


,totalEnergyUse
Property Id,
10417930,726554.8
8854298,406124.1
21988620,396268.1
21988617,245215.1
9492075,122386.6


,Property Id,Property Name,Address 1,City,Postal Code,Province,Primary Property Type - Self Selected,Number of Buildings,Year Built,Property GFA - Self-Reported (m²),...,Source EUI (GJ/m²),Weather Normalized Source EUI (GJ/m²),Total GHG Emissions (Metric Tons CO2e),Total GHG Emissions Intensity (kgCO2e/m²),Direct GHG Emissions (Metric Tons CO2e),Direct GHG Emissions Intensity (kgCO2e/m²),Electricity Use - Grid Purchase (kWh),Natural Gas Use (GJ),Year Ending,Unique ID
10,21988596,Calgary Soccer Centre and Annex,7000 48 ST SE,Calgary,T2C 4E1,Alberta,Indoor Arena,2,1990,16258.0,...,1.27,1.41,1372.0,84.4,609.0,37.4,1231244.0,11852.0,2023,2023-21988596
101,9498648,Jimmie Condon and Rose Kohn Arenas,502 Heritage Dr SW,Calgary,T2V 2W2,Alberta,Ice/Curling Rink,1,1969,5260.1,...,3.06,3.02,1195.2,227.2,398.7,75.8,1171212.2,7765.3,2019,2019-9498648
145,10417930,Stoney Transit Facility,1921 128 Ave NE,Calgary,T3K 0S5,Alberta,Distribution Center,1,2018,44228.3,...,1.83,1.83,7768.2,104.9,3518.8,79.6,1650567.6,68527.2,2019,2019-10417930
155,21988596,Calgary Soccer Centre and Annex,7000 48 ST SE,Calgary,T2C 4E1,Alberta,Indoor Arena,2,1990,16258.0,...,1.68,1.64,1372.0,111.2,923.1,56.8,1301988.5,17977.0,2019,2019-21988596
244,10417930,Stoney Transit Facility,1921 128 Ave NE,Calgary,T3K 0S5,Alberta,Distribution Center,1,2018,44228.3,...,2.41,2.41,7768.2,130.4,5012.7,113.3,1112911.2,97619.2,2020,2020-10417930
254,21988596,Calgary Soccer Centre and Annex,7000 48 ST SE,Calgary,T2C 4E1,Alberta,Indoor Arena,2,1990,16258.0,...,1.38,1.44,1372.0,90.8,773.1,47.6,1035014.0,15055.3,2020,2020-21988596
293,10417930,Stoney Transit Facility,1921 128 Ave NE,Calgary,T3K 0S5,Alberta,Distribution Center,1,2018,44228.0,...,5.91,5.89,7768.2,295.4,12243.0,276.8,1329838.0,238415.0,2023,2023-10417930
372,10417930,Stoney Transit Facility,1921 128 Ave NE,Calgary,T3K 0S5,Alberta,Distribution Center,1,2018,44228.3,...,3.39,3.39,7768.2,175.6,7450.2,168.4,467614.2,145089.0,2021,2021-10417930
383,21988596,Calgary Soccer Centre and Annex,7000 48 ST SE,Calgary,T2C 4E1,Alberta,Indoor Arena,2,1990,16258.0,...,1.18,1.20,1372.0,78.9,631.6,38.9,957222.5,12300.8,2021,2021-21988596
457,10417930,Stoney Transit Facility,1921 128 Ave NE,Calgary,T3K 0S5,Alberta,Distribution Center,1,2018,44228.0,...,3.94,3.94,7768.2,201.7,7952.0,179.8,1562154.0,154863.0,2022,2022-10417930


np.float64(-527.6249999999999)

np.float64(1182.975)

,medianGHGbyType
Primary Property Type - Self Selected,
Distribution Center,7768.2000
Fire Station,140.5000
Fitness Center/Health Club/Gym,785.6750
Heated Swimming Pool,693.7000
Ice/Curling Rink,908.7125
Indoor Arena,1372.0000
Mixed Use Property,65.0000
Museum,287.1000
Non-Refrigerated Warehouse,550.9000


,Property Id,Property Name,Address 1,City,Postal Code,Province,Primary Property Type - Self Selected,Number of Buildings,Year Built,Property GFA - Self-Reported (m²),...,Source EUI (GJ/m²),Weather Normalized Source EUI (GJ/m²),Total GHG Emissions (Metric Tons CO2e),Total GHG Emissions Intensity (kgCO2e/m²),Direct GHG Emissions (Metric Tons CO2e),Direct GHG Emissions Intensity (kgCO2e/m²),Electricity Use - Grid Purchase (kWh),Natural Gas Use (GJ),Year Ending,Unique ID
0,10176804,Acadia Aquatic & Fitness Centre,9009 Fairmount Dr SE,Calgary,T2H 0Z4,Alberta,Heated Swimming Pool,1,2010,1838.0,...,4.47,4.91,509.00,276.7,280.0,152.4,368616.0,5453.0,2023,2023-10176804
1,6169481,Ad Valorem,2924 11 ST NE,Calgary,T2E 7L7,Alberta,Office,1,1981,7770.0,...,1.57,1.64,920.00,118.4,243.0,31.2,1091854.0,4725.0,2023,2023-6169481
2,6305956,Alberta Trade Centre,315 10 AV SE,Calgary,T2G 0W2,Alberta,Office,1,1974,6681.0,...,0.94,1.00,499.00,74.6,0.0,0.0,508589.0,3.0,2023,2023-6305956
3,8854296,Andrew Davison,133 6 AV SE,Calgary,T2G 0G2,Alberta,Office,1,1979,17468.0,...,0.97,1.04,349.05,74.0,26.0,1.5,1152308.0,506.0,2023,2023-8854296
4,21988603,Animal Services Centre,2201 PORTLAND ST SE,Calgary,T2G 4M7,Alberta,Office,1,2000,2143.0,...,4.00,4.27,568.00,265.2,253.0,118.1,508594.0,4926.0,2023,2023-21988603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,21988624,Senior Services,311 34 AV SE,Calgary,T2G 1T7,Alberta,Office,1,1960,768.0,...,1.57,1.64,66.00,86.3,50.0,65.6,25615.0,980.0,2022,2022-21988624
490,21988625,Fire Station #25,4705 76 AV SE,Calgary,T2C 2X1,Alberta,Fire Station,1,1982,1108.0,...,1.49,1.49,105.00,94.8,53.0,48.2,83298.0,1040.0,2022,2022-21988625
491,21988627,EMS #6,1940 WESTMOUNT BV NW,Calgary,T2N 3G8,Alberta,Other - Public Services,1,1964,425.0,...,2.27,2.37,53.00,124.8,40.0,94.5,20725.0,782.0,2022,2022-21988627
492,21988628,Fire Station #24,2607 106 AV SW,Calgary,T2W 4H7,Alberta,Fire Station,1,1978,1108.0,...,1.37,1.35,105.00,94.3,41.0,36.8,102861.0,794.0,2022,2022-21988628
